Exercise 2
---


Let's goooo

In [43]:
import bootcamp_utils

#### 2.1.b

In [2]:
def reader(file_name):
    """A function to read a FASTA file and return the descriptor line and whole sequence as strings"""
    with open(file_name, 'r') as f:
        seq = ''
        for line in f:
            
            #if any line matches > as its first character, save that as descriptor line. Else, add line to seq variable
            if line[0] == ">":
                desc = line
            else:
                seq += line.rstrip("\n")
    return desc, seq

        
#file_name = '../../git/bootcamp/data/salmonella_spi1_region.fna'
#reader(file_name)
#print ("done")

#### 2.2.a

In [3]:
file_name = 'lambda_NEB_seq.txt'
reader(file_name)
print ("done")

done


#### 2.2.b

In [8]:
HindIII_req = 'AAGCTT'
EcoRI_req = 'GAATTC'
Kpnl_req = 'GGTACC'


def restriction_sites(seq, recog_seq):
    """Function to search for restriction sites in the given sequence and return a list of indices of the first base of the restriction seq when found"""
    i = 0
    indices=[]
    
    # if recognition sequence exists is equal to any segment of the sequence, with the same length, append its index to indices.
    while i < len(seq): 
        if recog_seq == seq[i:i+len(recog_seq)]:
            #print (i, seq_list[i:i+len(recoq_seq)])
            indices.append(i)
        i += 1
    return indices
name_seq, seq = reader(file_name)
restriction_sites(seq, Kpnl_req)

[17052, 18555]

#### 2.3.a

In [9]:
def gc_blocks(seq,block_size):
    """calculate for each block of sequence the gc content in fractions"""
    i = 0
    gc_content=[]
    
    #making sure that I get a uniform lower case sequence as input to the following section.
    seq_low=seq.lower()
    while i < len(seq_low):
        
        # making sure that I don't get an error what says Index is out of range. Break while loop if that happens
        if i + block_size > len(seq_low):
            break
        
        #if not broken, block is now a temporary segment of sequences, whose length is dictated by block_size. 
        #Doing the math right inside append() method and appending the result to gc_content list.
        else:
            block = seq_low[i:i+block_size]
            #print (block)
            gc_content.append((block.count('g') + block.count('c'))/block_size)
        i += block_size
    return tuple(gc_content)
    
    
seq = 'ATGACTACGTGGGGGGGGGGGGAA'
block_size = 4

gc_blocks(seq, block_size)

(0.25, 0.5, 0.75, 1.0, 1.0, 0.5)

#### 2.3.b

In [10]:
def gc_map(seq, block_size, gc_thresh):
    """use gc_content to conver the seq file into a mapped seq file"""
    # I wanted to incorporate gc_content function into this function so I started from it. Idea is to go through every block and if the block's gc_content
    # is lower or higher than threshold, calculate the corresponding sequence of that block from the master seq variable and upper() or lower() it and store in a string
    gc_cont = gc_blocks(seq, block_size)
    i = 0
    mapped = ''
    while i < len(gc_cont):
        
        #the seq[i*block_size : i*block_size+block_size] segmenter makes sure that I select the correct corresponding sequence segment for every block in gc_cont
        if gc_cont[i] < gc_thresh:
            block = seq[i*block_size:i*block_size+block_size].lower()
            #print (block, i)
        else:
            block = seq[i*block_size:i*block_size+block_size]
        mapped += block 
        #print ('block:', i)
        i += 1
    return mapped

seq = 'ATGACTACGTAGATTAGCGAACTT'
block_size = 5
gc_thresh = 0.4
print (len(seq))
gc_map(seq, block_size, gc_thresh)

24


'ATGACTACGTagattAGCGA'

#### 2.3.c

In [11]:
#using gc_map function to map the salmonella subregion.
file_name = '../../git/bootcamp/data/salmonella_spi1_region.fna'
block_size = 1000
gc_thresh = 0.45
desc, seq = reader(file_name)
#print (_ , len(seq))
salmo_seq_mapped = gc_map(seq, block_size, gc_thresh)
#len(salmo_seq_mapped)

#### 2.3.d

In [12]:
# writing descriptor into file with newline char at evry 60 line
with open('salmonella_spi1_region_mapped.fna', 'w') as f:
    f.write(desc)
    i=0
    while i < len(salmo_seq_mapped):
        f.write(salmo_seq_mapped[i:i+60])
        f.write("\n")
        i+=60

#### 2.4.a

In [133]:
def segger(seq):
    """partition the sequence into 3 reading frames, made it to use for longest_orf function. """
    frames=[]
    k = 0
    
    #3 loops for 3 reading frames.
    while k < 3:
        i = k
        frame=[]
        #frame.append(seq[:i])
        
        #if the length of the segment is 3, it is a complete codon (not at any of the extremities.). This way I don't end up with 1 or 2 letter code as a codon.
        # append to frame. frame is a list of codons of the current reading frame, covering the full sequence. This will be later appended fully to frames
        
        while i < len(seq):
            if len(seq[i:i+3]) == 3:
                frame.append(seq[i:i+3])
            i += 3
            
        # frames will contain the 3 reading frames. 
        frames.append(frame)
        k += 1
    return frames

seq = 'ATGACTACGTAGATTAGCGAACTT'
segger(seq)    

[['ATG', 'ACT', 'ACG', 'TAG', 'ATT', 'AGC', 'GAA', 'CTT'],
 ['TGA', 'CTA', 'CGT', 'AGA', 'TTA', 'GCG', 'AAC'],
 ['GAC', 'TAC', 'GTA', 'GAT', 'TAG', 'CGA', 'ACT']]

In [134]:
#making a tuple of start and stop codons
start_codon = ('ATG')
stop_codon = ('TGA','TAG','TAA')

def longest_orf(seq):
    """ For each frame, find every ORF and store them in a master list, containing ORF strings. 
        Then at the end find the longest ORF and print it out."""
    #Making sure I get a uniform sequence, all uppercase. in case we use a mapped sequence
    seq = seq.upper()
    
    # 3 loops for the 3 reading frames. Current frame list is stored in frame_use.
    j = 0
    orf_list = []
    while j < 3:
        orf = ''
        frame_use = segger(seq)[j]
        #print ("frame_use length", len(frame_use), len(seq))
        
        #i is index of the current codon in frame_use. This is needed later to define k's value. For every codon, check if codon is a start codon
        # if yes, define k with i. A while loop will go through and store the open reading frames.
        for i, codon in enumerate(frame_use):
            #|print (j, i, codon)
            if codon in start_codon:
                codon_sub = codon
                orf = ''
                k = i
                #print ("orf", orf, k)
                
                #start a loop to go through every codon after the current codon, which is a start codon. Check every frame if it's a stop codon.
                # as long as it doesn't encounter a stop codon, loop continues and frame_use[k] is stored in orf. If it encounters a stop codon, break.
                while k <= len(frame_use):
                    if frame_use[k] in stop_codon:
                        orf += frame_use[k]
                        break
                    orf += frame_use[k]
                    k += 1
                    #print ("k here", k)
                    
                # append every orf to a master orf_list.
                orf_list.append(orf)
        j += 1
    return max(orf_list, default="None found", key=len)

seq1 = 'ATGACTACGTAATAGATTAGCTGAGAACTT'
seq2 = 'ATGACTACGTGGATGAATGTAGATTAGCTAGTGAGAACTT'
seq3 = 'GGATGATGATGTAAAAC'
longest_orf(seq3)

'ATGATGATGTAA'

In [39]:
#testing the built-in max function
seq1 = ['ATGACTACGTAATAGATTAGCTGAGAACTT','ATGAAACTGAGAACTT', 'ATGATGATGATG']
max(seq1, key=len)
max(longest_orf(seq2), default='None found', key=len)

'A'

#### 2.4.b

In [40]:
# specifying file name and executing function
file_name = '../../git/bootcamp/data/salmonella_spi1_region.fna'
desc, seq = reader(file_name)

longest_orf(seq)

'ATGACCAACTACAGCCTGCGCGCACGCATGATGATTCTGATCCTGGCCCCGACCGTCCTGATAGGTTTGCTGCTCAGTATCTTTTTTGTAGTGCACCGCTATAACGACCTGCAGCGTCAACTGGAAGATGCCGGCGCCAGTATTATTGAACCGCTCGCCGTCTCCAGCGAATATGGTATGAACTTACAAAACCGGGAGTCTATCGGCCAACTTATCAGCGTCCTGCACCGCAGACACTCTGATATTGTGCGGGCGATTTCCGTTTATGACGATCATAACCGTCTGTTTGTAACGTCTAATTTCCATCTGGACCCCTCACAGATGCAGCTTCCCGCCGGAGCGCCGTTTCCACGTCGTCTGAGCGTTGATCGCCACGGCGATATTATGATTCTGCGCACCCCAATTATCTCGGAGAGCTATTCGCCGGACGAGTCAGCGATTGCTGACGCGAAAAATACCAAAAATATGCTGGGGTATGTGGCGCTTGAACTGGATCTCAAGTCGGTCAGGCTACAGCAATACAAAGAGATTTTTATCTCCAGCGTGATGATGCTTTTTTGTATTGGCATTGCGCTGATCTTTGGCTGGCGGCTTATGCGCGATGTCACCGGGCCTATCCGTAATATGGTGAATACCGTTGACCGCATTCGCCGCGGACAACTGGATAGCCGGGTGGAAGGATTTATGCTGGGCGAACTGGATATGCTGAAAAACGGCATTAATTCCATGGCGATGTCGCTTGCCGCCTATCACGAAGAGATGCAGCATAATATCGATCAGGCCACTTCGGACCTGCGTGAAACCCTTGAGCAGATGGAAATCCAAAACGTTGAGCTGGATCTGGCGAAAAAGCGTGCCCAGGAAGCGGCGCGTATTAAGTCGGAGTTCCTGGCGAACATGTCGCACGAACTGCGAACGCCGCTGAACGGCGTCATTGGCTTTACCCGCCTGACATTAAAAACGGAGCTGAATCCCACCCAGCGCGACCATCTGAACACC

#### 2.4.c

In [42]:
#making a dictionary of codon to one letter amino acid
aa_dict=dict(zip(bootcamp_utils.codon_list, bootcamp_utils.amino_acids))

def translator(seq):
    """DNA seq to protein seq translator, using dictionary built from bootcamp.utils"""
    
    #make every 3 base go through the dictionary to get its amino acid assigned and save it in prot_seq string
    i = 0
    prot_seq = ''
    while i < len(seq):
        prot_seq += aa_dict[seq[i:i+3]]
        i += 3
    return prot_seq
        
translator(longest_orf(seq))

'MTNYSLRARMMILILAPTVLIGLLLSIFFVVHRYNDLQRQLEDAGASIIEPLAVSSEYGMNLQNRESIGQLISVLHRRHSDIVRAISVYDDHNRLFVTSNFHLDPSQMQLPAGAPFPRRLSVDRHGDIMILRTPIISESYSPDESAIADAKNTKNMLGYVALELDLKSVRLQQYKEIFISSVMMLFCIGIALIFGWRLMRDVTGPIRNMVNTVDRIRRGQLDSRVEGFMLGELDMLKNGINSMAMSLAAYHEEMQHNIDQATSDLRETLEQMEIQNVELDLAKKRAQEAARIKSEFLANMSHELRTPLNGVIGFTRLTLKTELNPTQRDHLNTIERSANNLLAIINDVLDFSKLEAGKLILESIPFPLRNTLDEVVTLLAHSSHDKGLELTLNIKNDVPDNVIGDPLRLQQVITNLVGNAIKFTESGNIDILVEKRALSNTKVQIEVQIRDTGIGIPERDQSRLFQAFRQADASISRRHGGTGLGLVITQKLVNEMGGDISFHSQPNRGSTFWFHINLDLNPNVIIDGPSTACLAGKRLAYVEPNATAAQCTLDLLSDTPVEVVYSPTFSALPLAHYDIMILSVPVTFREPLTMQHERLAKAASMTDFLLLALPCHAQINAEKLKQGGAAACLLKPLTSTRLLPALTEYCQLNHHPEPLLMDTSKITMTVMAVDDNPANLKLIGALLEDKVQHVELCDSGHQAVDRAKQMQFDLILMDIQMPDMDGIRACELIHQLPHQQQTPVIAVTAHAMAGQKEKLLSAGMNDYLAKPIEEEKLHNLLLRYKPGANVAARLMAPEPAEFIFNPNATLDWQLALRQAAGKPDLARDMLQMLIDFLPEVRNKIEEQLVGENPNGLVDLVHKLHGSCGYSGVPRMKNLCQLIEQQLRSGVHEEELEPEFLELLDEMDNVAREAKKILG*'

MULTISPECIES: two-component sensor histidine kinase BarA [Salmonella]

#### 2.4.e

In [139]:
start_codon = ('ATG')
stop_codon = ('TGA','TAG','TAA')

#almost the same function as above. Since I made a master list of all orfs, I can sort through them to pick the n longest orfs and print them out.
# second argument n is now required. n is the number of longest orfs wanted by user.
def longest_orf_n(seq,n):
    """ For each frame, find every ORF and store them in a master list, containing ORF strings. 
        Then at the end find the 5 longest ORF and print it out."""
    seq = seq.upper()
    j = 0
    orf_list = []
    while j < 3:
        #orf_list = []
        orf = ''
        
        frame_use = segger(seq)[j]
        #print ("frame_use length", len(frame_use), len(seq))
        for i, codon in enumerate(frame_use):
            #|print (j, i, codon)
            if codon in start_codon:
                codon_sub = codon
                orf = ''
                k = i
                #print ("orf", orf, k)
                while k < len(frame_use)-1:
                    if frame_use[k] in stop_codon:
                        orf +=frame_use[k]
                        break
                    orf += frame_use[k]
                    k += 1
                    #print ("k here", k)
                orf_list.append(orf)
        j += 1
        
    # At this point the orf_list contains all orfs. 
    # Now to sort through and pick the longest ones. Final return variable is a master list maxn.
    p = 0
    orf_tuple = tuple(orf_list)
    
    # .sort(reverse=True, key=len) sorts the list in order of the longest length string to shortest. Most important part
    
    orf_list.sort(reverse=True, key=len)
    #print (orf_list)
    maxn =[]
    
    #Just some handler attempts for any bad input. If it's not a number, raise error. If the n value is longer than the # of total orf, take n as total # of orfs
    if not int(n): 
        print ("Not an integer.")
        raise
    elif int(n) > len(orf_list):
        print("Too long. showing all orfs")
        n = len(orf_list) 
        
    # Append n elements of orf_list (which is now in longest to shortest order) to maxn master list.
    while p < int(n):
        maxn.append(orf_list[p])
        p += 1
    return maxn

seq1 = 'ATGACTACGTAATAGATTAGCTGAGAACTT'
seq2 = 'ATGTGAACTACGTGGATGAATGTAGATTAGCTAGTGAGAACTT'
seq3 = 'GGATGATGATGTAAAAC'
#print(longest_orf_5(seq2))

In [142]:
file_name = '../../git/bootcamp/data/salmonella_spi1_region.fna'
desc, seq = reader(file_name)

#prompt user for desired input of n. assign the returned master list to maxn.
n = input('Please input the value of n:')
maxn = longest_orf_n(seq,n)

#write a header text and write every element of maxn into output file    
with open('exercise2_result.txt', 'w') as f:
    f.write("Results of exercise 2.4.e\n")
    i = 0
    for i in range(int(n)):
        f.write('Protein:' + str(i+1) + "\n" + translator(maxn[i]) + "\n")
    

Please input the value of n: 5
